In [ ]:
import numpy as np
import sep
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.ndimage import median_filter

from photutils.datasets import make_100gaussians_image
from photutils.background import Background2D, MedianBackground

from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel

from plotimg import plot_images
from lvmagp.images import Image
from lvmagp.images.processors.detection import DaophotSourceDetection, SepSourceDetection
from lvmagp.focus.focusseries import ProjectionFocusSeries, PhotometryFocusSeries

from photutils.centroids import centroid_quadratic

from lvmagp.focus.curvefit import fit_hyperbola

In [ ]:
from lvmtipo.actors import lvm
from lvmagp.images import Image
from plotimg import plot_images

await lvm.sci.pwi.start()
await lvm.sci.pwi.status()
await lvm.sci.foc.start()
await lvm.sci.foc.status()

await lvm.sci.agc.start()


In [ ]:
deblend_nthresh = 1.4

source_detection = SepSourceDetection(threshold= 3.0, minarea=14.0, deblend_nthresh=deblend_nthresh)
source_detection = SepSourceDetection(threshold= 3.0, deblend_nthresh=deblend_nthresh)

num_stars = 17
focus = 41.2

await lvm.sci.pwi.status()

stars={}
fwhm={}

data = [[],[]]
column = "fwhm"

for foc in np.linspace(focus-2.0, focus+3.0, num=7):
    print(foc)
    await lvm.sci.foc.moveAbsolute(foc, "DT")    
    filenames = (await lvm.sci.agc.expose(2.0)).flatten().unpack("*.filename") 
#    print(filenames)
    images = [Image.from_file(f) for f in filenames]

    for idx, img in enumerate(images):
        images[idx].data = median_filter(images[idx].data, size=2)
        
        # photutils.background
#        bkg_estimator = MedianBackground()
#        bkg = Background2D(img.data.astype(float), (50, 50), filter_size=(3, 3),
#                           bkg_estimator=bkg_estimator)
#        images[idx].data = images[idx].data.astype(float) - bkg.background
#        bkg_median = np.median(bkg.background)
#        bkg_std = np.std(bkg.background)

        # sep.background
        bkg = sep.Background(images[idx].data.astype(float))
        images[idx].data = images[idx].data.astype(float) - bkg
        bkg_median = np.median(bkg)
        bkg_std = np.std(bkg_median)
        
#        print(f"median {bkg_median}, std {bkg_std}")
   
    for idx, img in enumerate(images):
        images[idx] = await source_detection(img)
#        images[idx].catalog.sort("fwhm")
#        images[idx].catalog = img_sep.catalog[img_sep.catalog["peak"] > bkg_median*2]
#        images[idx].catalog = img_sep.catalog[img_sep.catalog["ellipticity"] < 0.35]
        images[idx].catalog.sort("flux")
        images[idx].catalog.reverse()

        sources = images[idx].catalog
        if (len(sources) > 2):
            radius = np.median(sources[column])
            radius_err = np.std(sources[column])
            data[idx].append({"focus": foc, "r": radius, "rerr": radius_err})

    plot_images(images, cat_max=num_stars, cat_rest=True,)

#print(data)
fig, ax = plt.subplots(1, ncols=(len(data)), figsize=(8, 5/len(data)))

for idx, dd in enumerate(data):
    focus = [d["focus"] for d in dd]
    r = [d["r"] for d in dd]
    rerr = [d["rerr"] for d in dd]

    ax_idx = ax[idx] if len(data) > 1 else ax
    ax_idx.set_title(images[idx].header["CAMNAME"])
    ax_idx.errorbar(focus, r, yerr=rerr)

    foc, err = fit_hyperbola(focus, r, rerr)
    print(f"focus: {foc}, {err}")
